# Pathos＋initializer+globals()
## うまくglobals()は渡せない
- with Pool(os.cpu_count()) as p:


In [1]:
import os
from multiprocessing import Pool, cpu_count 
from pathos.pools import ParallelPool
from pathos.pools import ProcessPool

from tqdm import tqdm 
from glob import glob
import numpy as np

import dill
import pickle
import pathos_global_test

from pathos_global_test import PathosTest
pathos_test = PathosTest()
top_results = [[0, 0] for _ in range(11)]
# pathos_test.top_namespace = globals()
print(f'global namespace = {pathos_test.top_namespace}')

# initializerの定義
# https://superfastpython.com/multiprocessing-pool-shared-global-variables/
def init_worker(globals_object):
    print(f'globals object = {globals_object}')
    pathos_test.top_namespace = globals_object

# wrap_save_npy.namespace_main = globals()
# print(wrap_save_npy.namespace_main)
# サブプロセス群に入力データを分配 (データ並列) して関数を並列実行する
# p = Pool(processes=cpu_count()-1)
pool = ParallelPool(initializer= init_worker, initargs=(globals(),), node=cpu_count()-1)
# pool = ParallelPool( node=cpu_count()-1)
# pool = ProcessPool( node=cpu_count()-1)
# NumPy Arrayをsaveするフォルダ
save_base_path = 'gaussian_filtered_nparrays/'
# フォルダ作成
if not os.path.exists(save_base_path):
    os.makedirs(save_base_path)
# pngファイル名リストを作成
path_list = glob('icopng/*.png')

print(f'globals = {pathos_test.top_namespace}')
# processに渡す引数の作成

job_args = [(path, path.split('.')[1], num, 'top_results') for num,path in enumerate(path_list)]
print(job_args[0])
# mapナシならglobals()は動作している
pathos_test.wrap_save_npy(job_args[0])


# imapによってprocessの順番は任意で処理される

# for result in p.imap_unordered(wrap_save_npy, job_args):
# for result in p.imap(wrap_save_npy, job_args):
# mapでシリアライズされるとglobals()を渡しても動作しなくなる
for result in pool.map(pathos_test.wrap_save_npy, job_args):
    print(result)
# print(f' top results = {top_results}')

global namespace = None
globals = None
('icopng\\icons8-キャンセル-64.png', 'png', 0, 'top_results')
wrap_save_npy top_namespace = None
args = ('icopng\\icons8-キャンセル-64.png', 'png', 0, 'top_results')


TypeError: 'NoneType' object is not iterable